# Testing the creation of a 'xr_phenology' function

Random online python phenology functions that might help:

- https://gist.github.com/YanCheng-go/d4e17831f294199443d0f7682558e608

- https://github.com/JavierLopatin/PhenoPY

**UPDATE 25-5-2020**
 - `xr_phenology` is now working on 1D xarrays, but fails on 3D arrays because the `da.sel(time=slice(etc))` method cannot select a different time slice per-pixel...
 - `xr_polyfit_smooth` works on 3D arrays.
     - Doesn't yet handle NaNs, and requires documenting. 
     - Time dimension is returned as a simple integer representing the DOY. 
     - Should consider mapping this back to proper dates...
     - how would the function handle a timeseries that went across a calender year?


### Load modules

In [1]:
import sys
import os
import datacube
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import warnings
import pandas as pd
import deafrica_phenology
sys.path.append('../Scripts')
from deafrica_plotting import display_map, rgb
from deafrica_datahandling import mostcommon_crs, load_ard
from deafrica_bandindices import calculate_indices
from deafrica_phenology import xr_phenology
from deafrica_dask import create_local_dask_cluster

%load_ext autoreload
%autoreload 2

/env/lib/python3.6/site-packages/datacube/storage/masking.py:4: DeprecationWarning: datacube.storage.masking has moved to datacube.utils.masking
  category=DeprecationWarning)


In [2]:
create_local_dask_cluster(aws_unsigned=False)

Client Scheduler: tcp://127.0.0.1:42247 Dashboard: /user/chad/proxy/8787/status,Cluster Workers: 1 Cores: 2 Memory: 14.18 GB


### Load some data to work with

In [3]:
dc = datacube.Datacube(app='phenology stats')

In [4]:
# lat = -10.6976
# lon = 35.2708
# lon_buffer = 0.00200
# lat_buffer = 0.00190

lat = 31.106 #30.2522
lon = 31.1385#30.5516
buffer = 0.15

x = (lon - buffer, lon + buffer)
y =  (lat + buffer, lat - buffer)

# Create a reusable query
query = {
    'x': x,
    'y': y,
    'time': ('2018-01', '2018-06'),
    'resolution': (-20, 20),
    'dask_chunks':{'x':500, 'y':500,'time': -1}
}

In [5]:
# display_map(x=x, y=y)

In [6]:
#find the most common UTM crs for the location
output_crs = mostcommon_crs(dc=dc, product='usgs_ls8c_level2_2', query=query)

# Load available data
ds = load_ard(dc=dc, 
              products=['usgs_ls8c_level2_2'],
              measurements=['red','nir'],
              group_by='solar_day',
              min_gooddata=0.5,
              output_crs=output_crs, 
              **query)

# Print output data
print(ds)

../Scripts/deafrica_datahandling.py:230: UserWarning: Setting 'min_gooddata' percentage to > 0.0 will cause dask arrays to compute when loading pixel-quality data to calculate 'good pixel' percentage. This can slow the return of your dataset.
  warnings.warn("Setting 'min_gooddata' percentage to > 0.0 "


Using pixel quality parameters for USGS Collection 2
Finding datasets
    usgs_ls8c_level2_2
Counting good quality pixels for each time step
Filtering to 10 out of 23 time steps with at least 50.0% good quality pixels
Applying pixel quality/cloud mask
Returning 10 time steps as a dask array
<xarray.Dataset>
Dimensions:      (time: 10, x: 1459, y: 1687)
Coordinates:
  * x            (x) float64 3.079e+05 3.079e+05 3.079e+05 ... 3.37e+05 3.37e+05
  * time         (time) datetime64[ns] 2018-01-12T08:29:43.179392 ... 2018-06-21T08:28:36.140511
  * y            (y) float64 3.46e+06 3.46e+06 3.46e+06 ... 3.426e+06 3.426e+06
    spatial_ref  int32 0
Data variables:
    red          (time, y, x) float32 dask.array<chunksize=(10, 500, 500), meta=np.ndarray>
    nir          (time, y, x) float32 dask.array<chunksize=(10, 500, 500), meta=np.ndarray>
Attributes:
    crs:           epsg:32636
    grid_mapping:  spatial_ref


### Calculate NDVI and DOY

In [ ]:
# rgb(ds, index=[0,2,4,6,8,10,12,14,16,18,20])

In [7]:
# First we calculate NDVI on each image in the timeseries
ndvi = calculate_indices(ds, index='NDVI', collection='c2', drop=True)
# ndvi = ndvi.NDVI

Dropping bands ['red', 'nir']


In [ ]:
ndvi.NDVI.mean(['x','y']).plot()

### Test each statistic

After confirming each statistic is working, then adding the code to the `deafrica_phenology.py` script.

Statistics to calculate:

    SOS = DOY of start of season
    POS = DOY of peak of season
    EOS = DOY of end of season
    vSOS = Value at start of season
    vPOS = Value at peak of season
    vEOS = Value at end of season
    LOS = Length of season (DOY)
    AOS = Amplitude of season (in value units)
    IOS = Integral of season (SOS-EOS)
    ROG = Rate of greening
    ROS = Rate of senescence
    SW = Skewness of growing season


Statistics that are working:

In [ ]:
vpos=deafrica_phenology._vpos(ndvi)
ipos = deafrica_phenology._ipos(ndvi)
pos = deafrica_phenology._pos(ndvi)
trough = deafrica_phenology._trough(ndvi)
aos = deafrica_phenology._aos(vpos, trough)
sos = deafrica_phenology._sos(ndvi, ipos, method_sos='first')
# vsos = deafrica_phenology._vsos(ndvi, sos)
# eos = deafrica_phenology._eos(ndvi, ipos, method_eos='last')
# veos = deafrica_phenology._veos(ndvi, eos)
# los = deafrica_phenology._los(eos, sos)
# ios = deafrica_phenology._ios(ndvi, sos, eos, dt_unit='D')
# rog = deafrica_phenology._rog(vpos,vsos,pos,sos)
# ros = deafrica_phenology._ros(veos,vpos,eos,pos)
# skew = deafrica_phenology._skew(ndvi, sos, eos)

In [ ]:
xr_phenology(ndvi.mean(['x','y']), stats=['SOS', 'POS','EOS','Trough','vSOS','vPOS',
                                 'vEOS', 'LOS', 'AOS','IOS','ROG','ROS', 'Skew'])

## Testing curve-fitting

In [10]:
def poly_fit(time, data, degree):
    
    pfit = np.polyfit(time, data, degree) 
    
    return np.transpose(np.polyval(pfit,time))

def poly_fit_smooth(time, data, degree, n_pts):
        """
        """
        time_smooth_inds = np.linspace(0, len(time), n_pts)
        time_smooth = np.interp(time_smooth_inds, np.arange(len(time)), time)

        data_smooth = np.array([np.array([coef * (x_val ** current_degree) for
                                coef, current_degree in zip(np.polyfit(time, data, degree),
                                range(degree, -1, -1))]).sum() for x_val in time_smooth])

        return data_smooth

def xr_polyfit(doy,
               da,
               degree,
               interp_multiplier=1):    
    
    # Fit polynomial curve to observed data points
    if interp_multiplier==1:
        print('Fitting polynomial curve to existing observations')
        pfit = xr.apply_ufunc(
            poly_fit,
            doy,
            da, 
            kwargs={'degree':degree},
            input_core_dims=[["time"], ["time"]], 
            output_core_dims=[['time']],
            vectorize=True,  
            dask="parallelized",
            output_dtypes=[da.dtype],
        )
    
    if interp_multiplier > 1:
        print("Fitting polynomial curve to "+str(len(doy)*interp_multiplier)+
                                                      " interpolated points")
        pfit = xr.apply_ufunc(
            poly_fit_smooth,  # The function
            doy,# time
            da,#.chunk({'time': -1}), #the data
            kwargs={'degree':degree, 'n_pts':len(doy)*interp_multiplier},
            input_core_dims=[["time"], ["time"]], 
            output_core_dims=[['new_time']], 
            output_sizes = ({'new_time':len(doy)*interp_multiplier}),
            exclude_dims=set(("time",)),
            vectorize=True, 
            dask="parallelized",
            output_dtypes=[da.dtype],
        ).rename({'new_time':'time'})
    
        # Map 'dayofyear' onto interpolated time dim
        time_smooth_inds = np.linspace(0, len(doy), len(doy)*interp_multiplier)
        new_datetimes = np.interp(time_smooth_inds, np.arange(len(doy)), doy)
        pfit = pfit.assign_coords({'time':new_datetimes})
    
    return pfit

In [13]:
%time
dayofyear = ndvi.time.dt.dayofyear.values
data = ndvi.NDVI
interp_multiplier = 2
degree = 4

pfit = xr_polyfit(dayofyear, 
                  data,
                  degree,
                  interp_multiplier)
pfit

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs
Fitting polynomial curve to 20 interpolated points


<xarray.DataArray 'NDVI' (y: 1687, x: 1459, time: 20)>
dask.array<vectorize_poly_fit_smooth, shape=(1687, 1459, 20), dtype=float64, chunksize=(500, 500, 20), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 3.079e+05 3.079e+05 3.079e+05 ... 3.37e+05 3.37e+05
  * y            (y) float64 3.46e+06 3.46e+06 3.46e+06 ... 3.426e+06 3.426e+06
    spatial_ref  int32 0
  * time         (time) float64 12.0 20.42 28.84 37.26 ... 171.2 172.0 172.0

In [14]:
%%time
pfit=pfit.compute()
pfit

distributed.nanny - WARNING - Restarting worker
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/env/lib/python3.6/site-packages/IPython/core/magics/execution.py", line 1312, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 1, in <module>
  File "/env/lib/python3.6/site-packages/xarray/core/dataarray.py", line 828, in compute
    return new.load(**kwargs)
  File "/env/lib/python3.6/site-packages/xarray/core/dataarray.py", line 802, in load
    ds = self._to_temp_dataset().load(**kwargs)
  File "/env/lib/python3.6/site-packages/xarray/core/dataset.py", line 654, in load
    evaluated_data = da.compute(*lazy_data.values(), **kwargs)
  File "/env/lib/python3.6/site-packages/dask/base.py", line 436, in compute
    results = schedule(dsk, keys, **kwargs)
  File "/env/lib/python3.6/site-packages/distributed/client.py", line 2572, in get
    results = self.gather(packed, asynchronous=asynchronous, direct=direct)
  File "/env/lib/python3.6/site-packages/distributed/client.py", line 1872, in gather
    asynchronous=asynchronous,

distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: 

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


In [ ]:
pfit.mean(['x','y']).plot()

In [ ]:
ndvi.NDVI.mean(['x','y']).plot()

---

#### This works...but only for 1D arrays. Could be good if used after zonal-stats are calculated

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
from scipy.integrate import trapz
from scipy.stats import skew

def _getPhenologyMetrics(da, doy):
    
    """
    Obtain land surfurface phenology metrics
    Parameters
    ----------
    - da:  xr.Datarray
    - doy: xt.DataArray
        Dayofyer values for each time step in the 'time'
        dim on 'da'. e.g doy=da.time.dt.dayofyear
    Outputs
    -------
        SOS = DOY of start of season
        POS = DOY of peak of season
        EOS = DOY of end of season
        vSOS = Value at start of season
        vPOS = Value at peak of season
        vEOS = Value at end of season
        LOS = Length of season (DOY)
        AOS = Amplitude of season (in value units)
        IOS = Integral of season (SOS-EOS)
        ROG = Rate of greening
        ROS = Rate of senescence
        SW = Skewness of growing season
    """
    stats=[]
    
    # basic variables
    vpos = np.nanmax(da)
    ipos = np.where(da == vpos)[0]
    print(ipos)
    pos = doy[ipos]
    trough = np.nanmin(da)
    ampl = vpos - trough

    # get position of seasonal peak and trough
    ipos = np.where(da == vpos)

    # scale annual time series to 0-1
    ratio = (da - trough) / ampl

    # separate greening from senesence values
    dev = np.gradient(ratio)  # first derivative
    greenup = np.zeros([ratio.shape[0]],  dtype=bool)
    greenup[dev > 0] = True

    # estimate SOS and EOS as median of the seasons
    i = np.nanmedian(doy[:ipos[0][0]][greenup[:ipos[0][0]]])
    ii = np.nanmedian(doy[ipos[0][0]:][~greenup[ipos[0][0]:]])
    sos = doy[(np.abs(doy - i)).argmin()]
    eos = doy[(np.abs(doy - ii)).argmin()]
    isos = np.where(doy == int(sos))[0]
    ieos = np.where(doy == eos)[0]
    if sos is None:
        isos = 0
        sos = doy[isos]
    if eos is None:
        ieos = len(doy) - 1
        eos = doy[ieos]

    # los: length of season
    los = eos - sos
    if los < 0:
        los[los < 0] = len(da) + \
            (eos[los < 0] - sos[los < 0])

    # doy of growing season
    green = doy[(doy > sos) & (doy < eos)]
    _id = []
    for i in range(len(green)):
        _id.append((doy == green[i]).nonzero()[0])

    # index of growing season
    _id = np.array([item for sublist in _id for item in sublist])
    # get intergral of green season
    ios = trapz(da[_id], doy[_id])

    # rate of greening [slope SOS-POS]
    rog = (vpos - da[isos]) / (pos - sos)
    rog = rog[0]

    # rate of senescence [slope POS-EOS]
    ros = (da[ieos] - vpos) / (eos - pos)
    ros= ros[0]

    # skewness of growing season
    sw = skew(da[_id])

    #values at start of season
    vsos = da[isos][0]

    #values at end of season
    veos = da[ieos][0]
    
    print(sos,pos[0],eos, vsos, vpos, veos, los, ampl, ios, rog, ros, sw)

    #return metrics

#create fake data
ndvi = np.array([0,0.0,0.2, 0.5, 0.9, 0.9, 0.9, 0.8, 0.75, 0.1, 0.0, 0.0])
test_da = xr.DataArray(ndvi,
             coords=[pd.date_range("01/01/2018", periods=12, freq=pd.DateOffset(months=1),)],dims="time")

doy=test_da.time.dt.dayofyear

xr.apply_ufunc(
        _getPhenologyMetrics,
        test_da,
        input_core_dims=[["time"]],
        kwargs={'doy': doy.values},
        dask='allowed')

ax = plt.subplot(1, 1, 1)
ax.plot(doy, ndvi)
ax.plot(60,0.2, "or")
ax.plot(244,0.75, "or")
ax.plot(121, 0.9, "or")